In [ ]:
import shutil
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import shutil
from yolo_tools import mseg_class_update, mseg2seg, random_select, ref_split, data_merge, seg_filter_and_remove, data_check

from yolo_mask_crop import myolo_crop
from classify_tools import ref_split_class
from yolo_sta import yolo_sta
from select_defect_data import obj_record, select_by_predict
from yolo_result_analysis import model_pred_compare


In [ ]:
defect_list = ['abandonment', 'broken', 'corrosion', 'deformation']
level_list = ['no', 'medium', 'high']

ps_src_dir = r'/localnvme/data/billboard/ps_data/yolo'
ps_dir = r'/localnvme/data/billboard/ps_data/psdata735'
ps_seg_dir = ps_dir + '_seg'
ps_seg_c6_dir = ps_dir + '_seg_c6'
ps_mseg_dir = ps_dir + '_mseg'
ps_mseg_c6_dir = ps_dir + '_mseg_c6'
ps_mseg_c6_update_dir = ps_dir + '_mseg_c6_update'


bd_seg_c6_dir = r'/localnvme/data/billboard/bd_data/data626_seg_c6'
bd_mseg_c6_dir = r'/localnvme/data/billboard/bd_data/data626_mseg_c6'
bd_mseg_c6_update_dir = r'/localnvme/data/billboard/bd_data/data626_mseg_c6_update'

fuse_seg_c6_dir = r'/localnvme/data/billboard/fused_data/data1361_seg_c6'
fuse_mseg_c6_dir = r'/localnvme/data/billboard/fused_data/data1361_mseg_c6'
fuse_mseg_c6_update_dir = r'/localnvme/data/billboard/fused_data/data1361_mseg_c6_update'

In [ ]:
if os.path.exists(ps_src_dir):
    os.rename(ps_src_dir, ps_mseg_dir)
mseg_class_update(ps_mseg_dir, ps_mseg_c6_dir)
mseg2seg(ps_mseg_c6_dir, ps_seg_c6_dir)

random_select(ps_mseg_c6_dir)
random_select(ps_seg_c6_dir)

In [ ]:
check_result_ps = data_check(ps_seg_c6_dir)
check_result_bd = data_check(bd_seg_c6_dir)

if len(check_result_ps+check_result_bd) == 0:
    data_merge(ps_seg_c6_dir,
               bd_seg_c6_dir,
               fuse_seg_c6_dir)
else:
    print(f'find {check_result_ps} and {check_result_bd}')

In [ ]:
seg_filter_and_remove(
    input_dir=ps_mseg_c6_dir,
    output_dir=ps_mseg_c6_update_dir,
    remove_class=6,
    cp_img=True,
    threshold=0.01,
    class_list=[1,3,5],
    with_attribute=True
)

In [ ]:
ref_split(
    ref_path = os.path.join(ps_mseg_c6_dir, 'val.txt'),
    img_dir = os.path.join(ps_mseg_c6_update_dir, 'images'),
    label_dir = os.path.join(ps_mseg_c6_update_dir, 'labels'),
    add_suffix='',
)

In [ ]:
attribute_path = os.path.join(bd_mseg_c6_dir, 'attribute.yaml')
check_result_ps1 = data_check(ps_mseg_c6_dir, mdet=True, attribute_path=attribute_path)
check_result_bd1 = data_check(bd_mseg_c6_dir, mdet=True, attribute_path=attribute_path)
check_result_ps2 = data_check(ps_mseg_c6_dir, mdet=True, attribute_path=attribute_path, check_item='attribute')
check_result_bd2 = data_check(bd_mseg_c6_dir, mdet=True, attribute_path=attribute_path, check_item='attribute')

if len(check_result_ps1+check_result_bd1 + check_result_ps2+check_result_bd2) == 0:
    data_merge(ps_mseg_c6_dir,
               bd_mseg_c6_dir,
               fuse_mseg_c6_dir)
    shutil.copy2(
        os.path.join(bd_mseg_c6_dir, 'class.txt'),
        os.path.join(fuse_mseg_c6_dir, 'class.txt'),
    )
    shutil.copy2(
        os.path.join(bd_mseg_c6_dir, 'attribute.yaml'),
        os.path.join(fuse_mseg_c6_dir, 'attribute.yaml'),
    )
else:
    print(f'find {check_result_ps1} and {check_result_bd1}')
    print(f'find {check_result_ps2} and {check_result_bd2}')

In [ ]:
attribute_path = os.path.join(bd_mseg_c6_dir, 'attribute.yaml')
check_result_ps1 = data_check(ps_mseg_c6_update_dir, mdet=True, attribute_path=attribute_path)
check_result_bd1 = data_check(bd_mseg_c6_update_dir, mdet=True, attribute_path=attribute_path)
check_result_ps2 = data_check(ps_mseg_c6_update_dir, mdet=True, attribute_path=attribute_path, check_item='attribute')
check_result_bd2 = data_check(bd_mseg_c6_update_dir, mdet=True, attribute_path=attribute_path, check_item='attribute')

if len(check_result_ps1+check_result_bd1 + check_result_ps2+check_result_bd2) == 0:
    data_merge(ps_mseg_c6_update_dir,
               bd_mseg_c6_update_dir,
               fuse_mseg_c6_update_dir)
    shutil.copy2(
        os.path.join(bd_mseg_c6_update_dir, 'class.txt'),
        os.path.join(fuse_mseg_c6_update_dir, 'class.txt'),
    )
    shutil.copy2(
        os.path.join(bd_mseg_c6_update_dir, 'attribute.yaml'),
        os.path.join(fuse_mseg_c6_update_dir, 'attribute.yaml'),
    )
else:
    print(f'find {check_result_ps1} and {check_result_bd1}')
    print(f'find {check_result_ps2} and {check_result_bd2}')

In [ ]:
dataset_dir = fuse_mseg_c6_dir
image_dir = os.path.join(dataset_dir, 'images')
labels_dir = os.path.join(dataset_dir, 'labels')
image_crop_dir = os.path.join(dataset_dir, 'images_crop')
class_file = os.path.join(dataset_dir, 'class.txt')
attribute_file = os.path.join(dataset_dir, 'attribute.yaml')
myolo_crop(image_dir, labels_dir, image_crop_dir, class_file,
           attribute_file=attribute_file, seg=True, annotation=True,
           save_method='attribute',
           crop_method='without_background_box_shape')

In [ ]:
dataset_dir = fuse_mseg_c6_update_dir
image_dir = os.path.join(dataset_dir, 'images')
labels_dir = os.path.join(dataset_dir, 'labels')
image_crop_dir = os.path.join(dataset_dir, 'images_crop')
class_file = os.path.join(dataset_dir, 'class.txt')
attribute_file = os.path.join(dataset_dir, 'attribute.yaml')
myolo_crop(image_dir, labels_dir, image_crop_dir, class_file,
           attribute_file=attribute_file, seg=True, annotation=True,
           save_method='attribute',
           crop_method='without_background_box_shape')

In [ ]:
dataset_dir = fuse_mseg_c6_dir
gt_dir=os.path.join(dataset_dir, "labels")
sta_dir=os.path.join(dataset_dir, "labels_sta")
class_path=os.path.join(dataset_dir, "class.txt")
attribute_path=os.path.join(dataset_dir, "attribute.yaml")
obj2img_path=os.path.join(dataset_dir, "images_crop.json")
sta_att_path=os.path.join(sta_dir, "sta_attribute.csv")
box_path=os.path.join(sta_dir, "sta_box.csv")
info_path=os.path.join(sta_dir, "info.csv")

In [ ]:
yolo_sta(
    img_dir=None,
    gt_dir=gt_dir,
    result_dir=sta_dir,
    class_path=class_path,
    attribute_path=attribute_path,
    seg=True,
)

In [ ]:
obj_record(sta_att_path, box_path, obj2img_path, info_path)

In [ ]:
dataset_dir = fuse_mseg_c6_update_dir

gt_dir=os.path.join(dataset_dir, "labels")
sta_dir=os.path.join(dataset_dir, "labels_sta")
class_path=os.path.join(dataset_dir, "class.txt")
attribute_path=os.path.join(dataset_dir, "attribute.yaml")
obj2img_path=os.path.join(dataset_dir, "images_crop.json")
sta_att_path=os.path.join(sta_dir, "sta_attribute.csv")
box_path=os.path.join(sta_dir, "sta_box.csv")
info_path=os.path.join(sta_dir, "info.csv")

In [ ]:
yolo_sta(
    img_dir=None,
    gt_dir=gt_dir,
    result_dir=sta_dir,
    class_path=class_path,
    attribute_path=attribute_path,
    seg=True,
)

In [ ]:
obj_record(sta_att_path, box_path, obj2img_path, info_path)

In [ ]:
dataset_dir = fuse_mseg_c6_dir
sta_dir=os.path.join(dataset_dir, "labels_sta")
info_path=os.path.join(sta_dir, "info.csv")
image_crop_dir = os.path.join(dataset_dir, 'images_crop')
val_path = os.path.join(dataset_dir, 'val.txt')
ref_split_class(ref_path=val_path,
                info_path=info_path,
                input_dir=image_crop_dir,
                defect_list = defect_list,
                level_list = level_list)


In [ ]:
dataset_dir = fuse_mseg_c6_update_dir
sta_dir=os.path.join(dataset_dir, "labels_sta")
info_path=os.path.join(sta_dir, "info.csv")
image_crop_dir = os.path.join(dataset_dir, 'images_crop')
val_path = os.path.join(dataset_dir, 'val.txt')
ref_split_class(ref_path=val_path,
                info_path=info_path,
                input_dir=image_crop_dir,
                defect_list = defect_list,
                level_list = level_list)

In [17]:
predict_dir = r'/localnvme/project/ultralytics/runs/msegment/val168/labels'
labels_dir = r'/localnvme/data/billboard/fused_data/data1361_mseg_c6_update/labels'
labels_npy_dir = r'/localnvme/project/ultralytics/runs/msegment/val168/labels_npy'
predict_npy_dir = r'/localnvme/project/ultralytics/runs/msegment/val168/predicts_npy'
model_pred_compare(labels_dir, predict_dir, seg=True, with_conf=False, with_att=True, threshold=0.5, att_num=4,image_width=1, image_height=1, labels_npy_dir=labels_npy_dir, predict_npy_dir=predict_npy_dir)

100%|██████████| 124/124 [00:08<00:00, 14.27it/s]

totally 379 box are counted


In [ ]:
image_dir = r'/localnvme/data/billboard/fused_data/data1361_mseg_c6_update/images'
compare_dir = r'/localnvme/data/billboard/fused_data/data1361_mseg_c6_update/labels_pred_compare'
output_dir = r'/localnvme/data/billboard/fused_data/data1361_mseg_c6_update/predict_val'
select_by_predict(compare_dir, image_dir, output_dir,
                  defect_list = defect_list,
                  level_list = level_list)